In [20]:
import torch
import torch.nn as nn


class Congfig:
    def __init__(self):
        self.ent_num = 14541
        self.rel_num = 237
        self.epochs = 300
        self.out_channels = 128
        self.dim = 100
        self.dropout_rate = 0.5
        self.lambd = 0.1
        self.learn_rate = 0.001
        self.batch = 100
        self.neg_num = 10


In [21]:
class ConvKB(nn.Module):
    def __init__(self, config):
        super(ConvKB,self).__init__()
        self.config = config
        self.device = device
        #进行实体和关系embedding
        self.ent_embeddings = nn.Embedding(config.ent_num, config.dim)
        self.rel_embeddings = nn.Embedding(config.rel_num, config.dim)
        self.bn_1 = nn.BatchNorm2d(1)
        self.conv = nn.Conv2d(1, config.out_channels, (1, 3))#采用1x3的卷积核
        self.bn_2 = nn.BatchNorm2d(self.config.out_channels)
        self.dropout1 = nn.Dropout(0.5)
        self.non_linearity = nn.ReLU()
        #全连接层，将dim*out_channel数据进行线性变换
        self.fc_layer1 = nn.Linear((config.dim)*config.out_channels,1)
        #定义损失函数
        self.criterion = nn.Softplus()
        
        #加载embedding的参数，如果没有则采用nn.init.xavier_uniform_方式进行初始化
    def init(self, ent2vec, rel2vec):
        self.ent_embeddings.weight.data.copy_(torch.from_numpy(ent2vec))
        self.rel_embeddings.weight.data.copy_(torch.from_numpy(rel2vec))
        nn.init.xavier_uniform_(self.fc_layer1.weight.data)
        #nn.init.xavier_uniform_(self.fc_layer2.weight.data)
        nn.init.xavier_uniform_(self.conv.weight.data)

    
    def _calc(self, h, r, t):
        h = h.unsqueeze(1)#batch*1*dim
        r = r.unsqueeze(1)
        t = t.unsqueeze(1)
        
        conv_input = torch.cat([h, r, t], dim = 1)# batch*3*dim
        #print(conv_input.shape)
        conv_input = conv_input.transpose(1, 2)#batch*dim*3
        conv_input = conv_input.unsqueeze(1)#batch*1*dim*3,增加维度1是因为CNN需要通道为1
        out_conv = self.bn_1(conv_input)
        out_conv = self.conv(out_conv)
        out_conv = self.bn_2(out_conv)
        out_conv = self.non_linearity(out_conv)
        out_conv = out_conv.view(-1,(self.config.dim)*self.config.out_channels)
        input_fc = self.dropout1(out_conv)
        score = self.fc_layer1(input_fc)
        score = score.squeeze(1)#注意这里，在调试时没有注意，导致训练出错，难以检测出来
        return score
    
    def loss(self, score, regul, labels):
        #损失函数，取反，使正列趋近于0，负例不断优化，逐渐变小
        return torch.mean(self.criterion(-1*score*labels))+self.config.lambd*regul
        
    
    def forward(self, h, r, t, labels):
        h_embed = self.ent_embeddings(h)
        r_embed = self.rel_embeddings(r)
        t_embed = self.ent_embeddings(t)
        
        score = self._calc(h_embed, r_embed, t_embed)
        
        #正则化
        l2_reg = torch.mean(h_embed**2) + torch.mean(r_embed**2) + torch.mean(t_embed**2)
        
        for W in self.conv.parameters():
            l2_reg += W.norm(2)
        for W in self.fc_layer1.parameters():
            l2_reg += W.norm(2)
        
        return self.loss(score,l2_reg, labels)  
    #预测函数，与_calc函数类似
    def predict(self, h, r, t):
        h_embed = self.ent_embeddings(h)
        r_embed = self.rel_embeddings(r)
        t_embed = self.ent_embeddings(t)
        
        
        score = self._calc(h_embed, r_embed, t_embed)
        return score.cpu().data.numpy()
            
        
        
        
        

In [22]:
import numpy as np
import random
import torch
import math

class Dataset:
    def __init__(self, ds_name):
        self.name = ds_name
        self.dir = "/home/qiupp/codestore/ConvKB/benchmarks/" + ds_name + "/"
        self.ent2id = {}
        self.rel2id = {}
        self.read_ent_id()
        self.read_rel_id()
        #print(self.ent2id, len(self.rel2id))
        self.data = {spl: self.read(self.dir + spl + ".txt") for spl in ["train2id", "valid2id", "test2id"]}
        self.batch_index = 0
        self.ent2vec, self.rel2vec = self.read_init()
       
    def read(self, file_path):
        with open(file_path, "r") as f:
            lines = f.readlines()
        
        triples = np.zeros((len(lines), 3))
        for i, line in enumerate(lines):
            temp = line.strip().split()
            #print(temp)
            if len(temp) == 1 :
                continue
            #print(temp)
            triples[i] = np.array(self.triple2ids(temp))
        return triples
    def read_init(self):
        with open(self.dir+"entity2vec100.init", 'r') as f:
            lines = f.readlines()
        ent2vec = []
        for line in lines:
            ent2vec.append(line.strip().split())
        ent2vec = np.array(ent2vec,dtype = np.float32)
        with open(self.dir+"relation2vec100.init", 'r') as f:
            lines = f.readlines()
        rel2vec = []
        for line in lines:
            rel2vec.append(line.strip().split())
        rel2vec = np.array(rel2vec, dtype = np.float32)
        return ent2vec, rel2vec
    
    def num_ent(self):
        return len(self.ent2id)
    
    def num_rel(self):
        return len(self.rel2id)
                     
    def triple2ids(self, triple):
        return [int(triple[0].strip()), int(triple[2].strip()), int(triple[1].strip())]
      
    def read_ent_id(self):
        with open(self.dir + 'entity2id.txt', 'r')as f:
            lines = f.readlines()
        for line in lines:
            temp = line.strip().split()
            if len(temp) <= 1:
                continue
            self.ent2id[temp[0]] = int(temp[1])
    
    def read_rel_id(self):
        with open(self.dir + 'relation2id.txt', 'r')as f:
            lines = f.readlines()
        for line in lines:
            temp = line.strip().split()
            if len(temp) <= 1:
                continue
            self.rel2id[temp[0]] = int(temp[1])
    def rand_ent_except(self, ent):
        rand_ent = random.randint(0, self.num_ent() - 1)
        while(rand_ent == ent):
            rand_ent = random.randint(0, self.num_ent() - 1)
        return rand_ent
                     
    def next_pos_batch(self, batch_size):
        if self.batch_index + batch_size < len(self.data["train2id"]):
            batch = self.data["train2id"][self.batch_index: self.batch_index+batch_size]
            self.batch_index += batch_size
        else:
            batch = self.data["train2id"][self.batch_index:]
            self.batch_index = 0
        return np.append(batch, np.ones((len(batch), 1)), axis=1).astype("int") #appending the +1 label
                     
    def generate_neg(self, pos_batch, neg_ratio):
        neg_batch = np.repeat(np.copy(pos_batch), neg_ratio, axis=0)
        for i in range(len(neg_batch)):
            if random.random() < 0.5:
                neg_batch[i][0] = self.rand_ent_except(neg_batch[i][0]) #flipping head
            else:
                neg_batch[i][2] = self.rand_ent_except(neg_batch[i][2]) #flipping tail
        neg_batch[:,-1] = -1
        return neg_batch

    def next_batch(self, batch_size, neg_ratio, device):
        pos_batch = self.next_pos_batch(batch_size)
        neg_batch = self.generate_neg(pos_batch, neg_ratio)
        batch = np.append(pos_batch, neg_batch, axis=0)
        np.random.shuffle(batch)
        heads  = torch.tensor(batch[:,0]).long().to(device)
        rels   = torch.tensor(batch[:,1]).long().to(device)
        tails  = torch.tensor(batch[:,2]).long().to(device)
        labels = torch.tensor(batch[:,3]).long().to(device)
        return heads, rels, tails, labels
    
    def was_last_batch(self):
        return (self.batch_index == 0)

    def num_batch(self, batch_size):
        return int(math.ceil(float(len(self.data["train2id"])) / batch_size))

In [23]:
datasets = Dataset("FB15K237")
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#print(datasets.next_batch(10, 1, device))
print(datasets.num_ent(), datasets.num_rel())
config = Congfig()

14541 237


In [24]:
import os
from tqdm import tqdm
class Trainer:
    def __init__(self, datasets, config, device):
        self.device = device
        self.datasets = datasets
        self.config = config
        self.model = ConvKB(config).to(device)
        self.model.init(datasets.ent2vec, datasets.rel2vec)
    def train(self):
        
        opimizer = torch.optim.Adagrad(self.model.parameters(),
                lr=self.config.learn_rate,
                weight_decay = 0)
        for epoch in tqdm(range(1, self.config.epochs+1)):
            tot_loss = 0.0
            lash_batch = False
            
            i = 0.0
            while not lash_batch:
                opimizer.zero_grad()
                lash_batch = self.datasets.was_last_batch()
                h, r, t, lables = self.datasets.next_batch(self.config.batch, self.config.neg_num, self.device)
                #print(lables)
                loss = self.model(h, r, t,lables)
                loss.backward()
                opimizer.step()
                tot_loss += loss.cpu().item()
                i += 1
                if i % 500 == 0:
                    print(loss.cpu().item())
            print("第 "+str(epoch)+" 轮损失函数： "+str(tot_loss))
            if epoch % 50 == 0:
                self.save_model(epoch)
    def save_model(self, epoch):
        directory = '/home/qiupp/codestore/ConvKB/models/'
        if not os.path.exists(directory):
            os.mkdir(directory)
        torch.save(self.model, directory+str(epoch)+".pkl")

        

In [25]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
trainer = Trainer(datasets, config, device)
trainer.train()

  0%|                                                                         | 0/300 [00:00<?, ?it/s]

第 1 轮损失函数： 1.653429627418518
0.6615503430366516
0.6041750907897949
0.5396155118942261
0.4843931496143341
0.4685586988925934


  1%|▍                                                                | 2/300 [00:20<50:18, 10.13s/it]

第 2 轮损失函数： 1565.7546453475952
0.4328857362270355
0.43700680136680603
0.3965681493282318
0.3782287538051605
0.37099435925483704


  1%|▋                                                              | 3/300 [00:41<1:12:26, 14.63s/it]

...

第 298 轮损失函数： 230.7273129746318
0.08653974533081055
0.10131055116653442
0.08892370760440826
0.08372065424919128
0.08759640902280807


100%|████████████████████████████████████████████████████████████▊| 299/300 [1:57:38<00:32, 32.92s/it]

第 299 轮损失函数： 230.1218115836382
0.08263865113258362
0.11379930377006531
0.08486193418502808
0.07811146974563599
0.08840574324131012


100%|█████████████████████████████████████████████████████████████| 300/300 [1:58:20<00:00, 23.67s/it]

第 300 轮损失函数： 230.40921949595213


In [32]:
class Measure:
    def __init__(self):
        self.hit1 = 0.0
        self.hit3 = 0.0
        self.hit10 = 0.0
        self.mr = 0.0
        self.mrr = 0.0
    def update(self, rank):
        if rank == 1:
            self.hit1 += 1.0
        if rank  <= 3:
            self.hit3 += 1.0
        if rank <= 10:
            self.hit10 += 1.0
        self.mr +=  rank
        self.mrr += (1.0/rank)
    def final_process(self, rel_num):
        self.hit1 /= rel_num
        self.hit10 /= rel_num
        self.hit3 /= rel_num
        self.mr /= rel_num
        self.mrr /= rel_num
    def print_(self):
        print("++++++++++++++++++++")
        print("hit1:   "+ str(self.hit1))
        print("hit3:   "+ str(self.hit3))
        print("hit10:   "+ str(self.hit10))
        print("mr:   "+ str(self.mr))
        print("mrr:   "+ str(self.mrr))
        

In [33]:
import numpy as np
class Tester:
    def __init__(self, datasets,model_path, device, valid_or_test):
        self.datasets = datasets
        self.all_facts = set(self.get_all_facts())
        self.device = device
        self.model = torch.load(model_path, map_location = self.device)
        self.model.eval()
        self.valid_or_test = valid_or_test
        self.measure = Measure()
        self.total = 2*len(datasets.data[valid_or_test])
    def get_all_facts(self):
        triples = []
        for sql in ['train2id', 'valid2id', 'test2id']:
            print(sql)
            for fact in self.datasets.data[sql]:
                triples.append(tuple(fact))
        return triples
    def add_rel_fact(self, fact, head_or_tail):
        h, r, t = fact
        
        add_triple = []
        if head_or_tail == 'head':
            for i in range(self.datasets.num_ent()):
                if i == h:
                    continue
                add_triple.append((i, r, t))
        elif head_or_tail == 'tail':
            for i in range(self.datasets.num_ent()):
                if i == t:
                    continue
                add_triple.append((h, r, i))
        
        return add_triple
    def move_repeat_triple(self,add_facts, fact):
        temp =  [tuple(fact)] + list( set(add_facts)- self.all_facts)
        return np.array(temp)
    
    def change_torch(self, add_triple):
        h = add_triple[:, 0]
        r = add_triple[:,1]
        t = add_triple[:, 2]
        return torch.LongTensor(h).to(self.device), torch.LongTensor(r).to(self.device), torch.LongTensor(t).to(self.device)
    def test(self):
        cn = 0
        for fact in self.datasets.data[self.valid_or_test]:
            for head_or_tail in ['head', 'tail']:
                new_facts = self.add_rel_fact(fact, head_or_tail)
                move_facts = self.move_repeat_triple(new_facts, fact)
                #print(len(move_facts))
                h,r,t = self.change_torch(move_facts)
                pred = self.model.predict(h, r, t)
            
                rank = (pred >= pred[0]).sum()
                cn += 1
                self.measure.update(rank)
        self.measure.final_process(self.total)
        print(cn)
        self.measure.print_()    

In [ ]:
model_path = '/home/qiupp/codestore/ConvKB/models/300.pkl'
Test = Tester(datasets, model_path, device,  'valid2id')
Test.test()

train2id
valid2id
test2id


[1 2 3 4 5]


10


tensor([-1., -2., -3.])


tensor(4.0764) tensor(4.7772)
